# Stream-DataSet Join Demo

### Stream-dataset joins

This has already been shown earlier while explain `DStream.transform` operation. Here is yet another example of joining a windowed stream with a dataset.
```python
dataset = ... # some RDD
windowedStream = stream.window(20)
joinedStream = windowedStream.transform(lambda rdd: rdd.join(dataset))
```
In fact, you can also dynamically change the `dataset` you want to join against. The function provided to `transform` is evaluated every batch interval and therefore will use the current dataset that `dataset` reference points to.

The complete list of DStream transformations is available in the API documentation. For the Python API, see [DStream](https://spark.apache.org/docs/latest/api/python/pyspark.streaming.html#pyspark.streaming.DStream).



### Demo

In [5]:
import findspark
# TODO: your path will likely not have 'jilg' in it. Change it to reflect your path.
findspark.init('/home/jilg/Downloads/spark-3.3.2-bin-hadoop3')

In [6]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
import time

In [8]:
#sc = SparkContext("local[2]", "IP-Matcher")
sc = SparkContext.getOrCreate("local[2]")

ssc = StreamingContext(sc, 2)

In [9]:
ips_rdd = sc.parallelize(set())
lines_ds = ssc.socketTextStream("localhost", 9999)

In [10]:
# split each line into IPs
ips_ds = lines_ds.flatMap(lambda line: line.split(" "))
pairs_ds = ips_ds.map(lambda ip: (ip, 1))

In [11]:
# join with the IPs RDD
matches_ds = pairs_ds.transform(lambda rdd: rdd.join(ips_rdd))
matches_ds.pprint()

In [12]:
# In another window run:
# $ nc -lk 9999
# Then enter IP addresses separated by spaces into the nc window

In [13]:
ssc.start()

# alternate between two sets of IP addresses for the RDD
IP_FILES = ('data/ip_file1.txt', 'data/ip_file2.txt')
file_index = 0
while True:
    with open(IP_FILES[file_index]) as f:
        ips = f.read().splitlines()
    ips_rdd = sc.parallelize(ips).map(lambda ip: (ip, 1))
    print("using" + IP_FILES[file_index])
    file_index = (file_index + 1) % len(IP_FILES)
    time.sleep(8)

usingdata/ip_file1.txt
-------------------------------------------
Time: 2023-02-19 14:56:06
-------------------------------------------

-------------------------------------------
Time: 2023-02-19 14:56:08
-------------------------------------------

-------------------------------------------
Time: 2023-02-19 14:56:10
-------------------------------------------

-------------------------------------------
Time: 2023-02-19 14:56:12
-------------------------------------------

usingdata/ip_file2.txt
-------------------------------------------
Time: 2023-02-19 14:56:14
-------------------------------------------

-------------------------------------------
Time: 2023-02-19 14:56:16
-------------------------------------------

-------------------------------------------
Time: 2023-02-19 14:56:18
-------------------------------------------

-------------------------------------------
Time: 2023-02-19 14:56:20
-------------------------------------------

usingdata/ip_file1.txt
-----------

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 44898)
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/jilg/Downloads/spark-3.3.2-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/jilg/Downloads/spark-3.3.2-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/jilg/Downloads/spark-3.3.2-bin-hadoop3/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


-------------------------------------------
Time: 2023-02-19 14:57:34
-------------------------------------------



Traceback (most recent call last):
  File "/usr/lib/python3.9/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/usr/lib/python3.9/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/usr/lib/python3.9/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/usr/lib/python3.9/socketserver.py", line 720, in __init__
    self.handle()
  File "/home/jilg/Downloads/spark-3.3.2-bin-hadoop3/python/pyspark/accumulators.py", line 281, in handle
    poll(accum_updates)
  File "/home/jilg/Downloads/spark-3.3.2-bin-hadoop3/python/pyspark/accumulators.py", line 253, in poll
    if func():
  File "/home/jilg/Downloads/spark-3.3.2-bin-hadoop3/python/pyspark/accumulators.py", line 257, in accum_updates
    num_updates = read_int(self.rfile)
  File "/home/jilg/Downloads/spark-3.3.2-bin-hadoop3/python/pyspark/serializers.py", line 5

Py4JError: An error occurred while calling o16.sc

In [ ]:
ssc.stop()

## Reference
1. https://spark.apache.org/docs/latest/streaming-programming-guide.html#join-operations